#Итоговый проект

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from os import listdir
import codecs
import pickle                                                                   
from bs4 import BeautifulSoup as BS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from boilerpipe.extract import Extractor
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
from scipy.sparse import hstack
from scipy.sparse import vstack
from scipy.sparse import hstack
from scipy.sparse import vstack
from scipy.sparse import csr_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
import re
%matplotlib inline

In [ ]:
submit = False

In [ ]:
def save_obj(obj, name ):                                                       
    with open(name + '.pkl', 'wb') as f:                                        
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)                            
                                                                                
def load_obj(name ):                                                            
    with open(name + '.pkl', 'rb') as f:                                        
        return pickle.load(f)       

In [12]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return ' '.join(res)

In [ ]:
name = 'NumWordsRulesExtractor'
#name = 'ArticleExtractor'
name = 'train_data'+name
train_data = load_obj(name)#Загружаем готовый распарсенный текст уже

In [4]:
#это парсинг теста и доп фич (meta)
if submit:
    test_data= pd.read_csv('test_groups.csv',dtype=np.int16)

    parser_type = 'NumWordsRulesExtractor'
    def save_obj(obj, name ):                                                       
        with open(name + '.pkl', 'wb') as f:                                        
            pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)                            

    def load_obj(name ):                                                            
        with open(name + '.pkl', 'rb') as f:                                        
            return pickle.load(f)                                                   
        
    description = {}                                                                      
    keywords= {}                                                                      
    test_data['description'] = ''
    test_data['keywords'] = ''

    path = 'content/'                                                               
    texts = {}                                                                      
    print('Start: ' + parser_type)                                                  
    test_data = pd.read_csv('test_groups.csv',dtype=np.int16)                      
    #for filename in ['6770.dat']:                                                  
    for filename in tqdm(listdir(path)):                                            
        doc_id = int(filename.strip('.dat'))                                        
        if doc_id not in test_data.doc_id.values:                                  
            continue                                                                
        try:                                                                        
            with codecs.open(path + filename, 'r', encoding='utf-8', errors='ignore') as f:
                url = f.readline().strip()                                          
                html = f.read()                                                     
                bs = BS(html, 'html.parser')#ищем след страницу                         
                meta = bs.find("meta", {"name":"description"})
                if meta:
                    description[doc_id] = meta.get('content','')
                meta = bs.find("meta", {"name":"keywords"})
                if meta:
                    keywords[doc_id] = meta.get('content','')
                extractor = Extractor(extractor=parser_type, html=html)             
                s = extractor.getText()                                             
                s=s.replace('\n'," ")                                               
                s=s.replace('\t'," ")                                               
                s=s.replace('\r'," ")                                               
                texts[doc_id] = s                                                   
        except UnicodeDecodeError: 
            print('error {}'.format(filename))                                      
            with codecs.open(path + filename, 'r', 'utf-8') as f:                   
                url = f.readline().strip()                                          
                html = f.read()                                                     
            bs = BS(html, 'html.parser')#ищем след страницу                         
            for s in bs.select('script'):                                           
                s.extract()                                                         
            s = bs.get_text()                                                       
            s=s.replace('\n',"")                                                    
            s=s.replace('\t',"")                                                    
            s=s.replace('\r',"")                                                    
            texts[doc_id] = s                                                       
    test_data['text'] = test_data.apply(lambda row: texts[row.doc_id], axis = 1)  
    test_data['description'] = test_data.apply(lambda row: description.get(row.doc_id,''), axis = 1)  
    test_data['keywords'] = test_data.apply(lambda row: keywords.get(row.doc_id,''), axis = 1)  
else:
    path = 'content/'                                                               
    description = {}                                                                      
    keywords= {}                                                                      
    train_data['description'] = ''
    train_data['keywords'] = ''
    #for filename in ['1.dat']:                                                  
    for filename in tqdm(listdir(path)):                                            
        doc_id = int(filename.strip('.dat'))                                        
        if doc_id not in train_data.doc_id.values:                                  
            continue                                                                
        with codecs.open(path + filename, 'r', encoding='utf-8', errors='ignore') as f:
            url = f.readline().strip()                                          
    #        urls[doc_id] = url                                                       
            html = f.read()                                                     
            bs = BS(html, 'html.parser')#ищем след страницу                         
            meta = bs.find("meta", {"name":"description"})
            if meta:
                description[doc_id] = lemmatize(meta.get('content',''))
            meta = bs.find("meta", {"name":"keywords"})
            if meta:
                keywords[doc_id] = lemmatize(meta.get('content',''))
    train_data['description'] = train_data.apply(lambda row: description.get(row.doc_id,''), axis = 1)  
    train_data['keywords'] = train_data.apply(lambda row: keywords.get(row.doc_id,''), axis = 1)  


Start: NumWordsRulesExtractor


error 6770.dat



In [8]:
len(description),len(keywords)

(11988, 8963)

In [ ]:
save_obj(train_data, name + 'lem_meta')

In [ ]:
str = train_data.urls[0]
import re
train_data['urls'] = train_data.apply(lambda row: re.split(';|,|\.|\_|\-|\*|\/|\&|\?',row.urls), axis = 1)  

In [9]:
doc_to_title = {}
with open('docs_titles.tsv', encoding = 'UTF-8') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
if submit:
    test_doc_titles = []
    for id in test_data.doc_id.values:
        test_doc_titles.append(doc_to_title[id])
    test_data['title'] = test_doc_titles
    print(len(test_doc_titles))
else:
    doc_titles = []
    for id in train_data.doc_id.values:
        doc_titles.append(doc_to_title[id])
    train_data['title'] = doc_titles
    print(len(doc_titles))


16627


In [193]:
test_data

NameError: name 'test_data' is not defined

In [ ]:
train_data.head()

In [13]:
if submit:
    test_data['lem_title'] = test_data.apply(lambda row:lemmatize(row.title) , axis = 1)  
else:
    train_data['lem_title'] = train_data.apply(lambda row:lemmatize(row.title) , axis = 1)  

In [15]:
if submit:
    test_data['sum_title'] = test_data.apply(lambda row: row.lem_title +row.description + row.keywords , axis = 1)  
else:
    train_data['sum_title'] = train_data.apply(lambda row: row.lem_title +row.description + row.keywords , axis = 1)  


In [ ]:
# train_data['urls'] = train_data.apply(lambda row:lemmatize(row.urls) , axis = 1)  

In [ ]:
# from tqdm.notebook import tqdm
# tqdm.pandas()

# # Now you can use `progress_apply` instead of `apply`

In [ ]:
#train_data['text'] = train_data.progress_apply(lambda row:lemmatize(row.text[0:1000]) , axis = 1)  
# train_data['text'] = train_data.progress_apply(lambda row: row.text[0:len(row.title)] , axis = 1)  

In [ ]:

# from tqdm.notebook import tqdm
# tqdm.pandas()
# train_data['text'] = train_data.progress_apply(lambda row: re.sub(r'\d+', '', row.text) , axis = 1)  

In [ ]:
def get_splits(data, n=3, random_state=None):
    groups = train_data.group_id.unique()
    kf = KFold(n_splits=n, shuffle=True, random_state=random_state)
    for i,j in kf.split(groups):
        train, test = data.group_id.isin(groups[i].reshape(-1)), data.group_id.isin(groups[j].reshape(-1))
        yield (train[train].index.values, test[test].index.values)

In [17]:
#vectorizer = CountVectorizer(ngram_range = (1,3))#max_features = 70)
stop_words =['без','в','вне','во','для','до','за','И','из','изо','к','ко','меж','на','над','о','об','обо','от','ото','по','под','при','про','с', 'а','же','и','ибо','или','как','но','раз','чем','что','не','ни','ли','ль','вот','вон','это','то','ну','как','же','ни','уж','ну','я', 'ты', 'он', 'мы', 'вы', 'её', 'их','кто','тот']

 
vectorizer = CountVectorizer()#max_features = 70)
#vectorizer = TfidfVectorizer(stop_words=stop_words)#max_features = 70)

In [18]:
all_groups = np.array(list(range(1,310))).reshape(-1,1)
i = OneHotEncoder()
i.fit(all_groups)
if submit:
    groups_one_hot_test   = i.transform(test_data.group_id.values.reshape(-1,1))
else:
    groups_one_hot_train = i.transform(train_data.group_id.values.reshape(-1,1))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [19]:
def get_top_n_words(corpus, n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.

    get_top_n_words(["I love Python", "Python is a language programming", "Hello world", "I love the world"]) -> 
    [('python', 2),
     ('world', 2),
     ('love', 2),
     ('hello', 1),
     ('is', 1),
     ('programming', 1),
     ('the', 1),
     ('language', 1)]
    """
    vec = CountVectorizer(stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

    return [i[0] for i in words_freq[:n]]

In [27]:
def get_features(data):
    scaler = StandardScaler(with_mean=False)
    number = 1
    data['first_pop'] =0
    for group in tqdm(data.group_id.unique()):
        number +=1
        elements_in_same_group = data.group_id == group
        #words = data.title[elements_in_same_group] + data.text[elements_in_same_group]
        words = data.lem_title[elements_in_same_group]
        vectorizer.fit(words)
        texts_vectorized_titles = vectorizer.transform(data.lem_title[elements_in_same_group])
        texts_vectorized_titles = scaler.fit_transform(texts_vectorized_titles)

        texts_vectorized_texts = vectorizer.transform(data.text[elements_in_same_group])
        texts_vectorized_texts = scaler.fit_transform(texts_vectorized_texts)
        texts_vectorized_keywords = vectorizer.transform(data.keywords[elements_in_same_group])
        texts_vectorized_keywords = scaler.fit_transform(texts_vectorized_keywords)
        texts_vectorized_description= vectorizer.transform(data.description[elements_in_same_group])
        texts_vectorized_description= scaler.fit_transform(texts_vectorized_description)
    
        texts_vectorized_sumtitles = vectorizer.transform(data.sum_title[elements_in_same_group])
        texts_vectorized_sumtitles = scaler.fit_transform(texts_vectorized_sumtitles)
        
        dist_matr = pairwise_distances(texts_vectorized_titles, texts_vectorized_titles, metric = 'cosine')
        dist_matr2 = pairwise_distances(texts_vectorized_texts, texts_vectorized_texts, metric = 'cosine')
        dist_matr3 = pairwise_distances(texts_vectorized_texts, texts_vectorized_titles, metric = 'cosine')
        dist_matr4 = pairwise_distances(texts_vectorized_keywords, texts_vectorized_keywords, metric = 'cosine')
        dist_matr5 = pairwise_distances(texts_vectorized_description, texts_vectorized_description, metric = 'cosine')
        dist_matr6 = pairwise_distances(texts_vectorized_sumtitles, texts_vectorized_texts, metric = 'cosine')
        dist_matr7 = pairwise_distances(texts_vectorized_keywords, texts_vectorized_titles, metric = 'cosine')
        dist_matr8 = pairwise_distances(texts_vectorized_description, texts_vectorized_titles, metric = 'cosine')

        #clustering = DBSCAN(eps = 1.3, min_samples=5).fit(dist_matr)
        #clf = IsolationForest(random_state=0).fit_predict(dist_matr)

        dist_matr_sorted = np.sort(dist_matr,axis=1)[:,0:21]
        dist_matr_sorted2 = np.sort(dist_matr2,axis=1)[:,0:21]
        dist_matr_sorted3 = np.sort(dist_matr3,axis=1)[:,0:21]
        dist_matr_sorted4 = np.sort(dist_matr4,axis=1)[:,:-1][:,0:21]
        dist_matr_sorted5 = np.sort(dist_matr5,axis=1)[:,:-1][:,0:21]
        dist_matr_sorted6 = np.sort(dist_matr6,axis=1)[:,:-1][:,0:21]
        dist_matr_sorted7 = np.sort(dist_matr7,axis=1)[:,:-1][:,0:21]
        dist_matr_sorted8 = np.sort(dist_matr7,axis=1)[:,:-1][:,0:21]
#         dist_matr_sorted = np.sort(dist_matr,axis=1)[:,0:min(50,dist_matr.shape[1])]
#         dist_matr_sorted2 = np.sort(dist_matr2,axis=1)[:,0:min(50,dist_matr2.shape[1])]
#         dist_matr_sorted3 = np.sort(dist_matr3,axis=1)[:,0:min(50,dist_matr3.shape[1])]

#         if dist_matr_sorted.shape[1] < 50:
#             dist_matr_sorted = np.hstack((dist_matr_sorted, np.tile(dist_matr_sorted[:,-1:], (1, 50-dist_matr_sorted.shape[1]))))
#             dist_matr_sorted2 = np.hstack((dist_matr_sorted2, np.tile(dist_matr_sorted2[:,-1:], (1, 50-dist_matr_sorted2.shape[1]))))
#             dist_matr_sorted3 = np.hstack((dist_matr_sorted3, np.tile(dist_matr_sorted3[:,-1:], (1, 50-dist_matr_sorted3.shape[1]))))


        mean_dist = dist_matr.mean(axis = 1)
        mean_dist2 = dist_matr2.mean(axis = 1)
        mean_dist3 = dist_matr3.mean(axis = 1)
        mean_dist4 = dist_matr4.mean(axis = 1)
        mean_dist5 = dist_matr5.mean(axis = 1)
        mean_dist6 = dist_matr6.mean(axis = 1)
        mean_dist7 = dist_matr7.mean(axis = 1)
        mean_dist8 = dist_matr8.mean(axis = 1)
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist2.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist3.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist4.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist5.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist6.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist7.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist8.reshape(-1,1)))
#         mean_dist = mean_dist *mean_dist 
#         mean_dist2 = mean_dist2 *mean_dist2 
#         mean_dist3 = mean_dist3 *mean_dist3 
#         mean_dist4 = mean_dist4 *mean_dist4 
#         mean_dist5 = mean_dist5 *mean_dist5 
#         mean_dist6 = mean_dist6 *mean_dist6 
#         mean_dist7 = mean_dist7 *mean_dist7 
#         dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist.reshape(-1,1)))
#         dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist2.reshape(-1,1)))
#         dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist3.reshape(-1,1)))
#         dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist4.reshape(-1,1)))
#         dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist5.reshape(-1,1)))
#         dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist6.reshape(-1,1)))
#         dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist7.reshape(-1,1)))
        
#         pop_words = get_top_n_words(words,n=100)
#         for pop_word in pop_words:
#             data.loc[elements_in_same_group,'first_pop']= data[elements_in_same_group].apply(lambda row: int(pop_word in row.text) , axis = 1)  
#             pop  = csr_matrix(data[elements_in_same_group].first_pop.values)
#             print(pop_word,pop.toarray())
#             dist_matr_sorted = np.hstack((dist_matr_sorted, pop.toarray().reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted2))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted3))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted4))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted5))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted6))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted7))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted8))
#        dist_matr_sorted = np.hstack((dist_matr_sorted, clustering.labels_.reshape(-1,1)))
#        dist_matr_sorted = hstack((dist_matr_sorted, texts_vectorized))

        if number == 2 :
            features_by_dist =  dist_matr_sorted
        else:
            features_by_dist = vstack((features_by_dist, dist_matr_sorted))
    return features_by_dist.tocsr()

In [28]:
if submit:
    features_by_dist = get_features(test_data)
    features_by_dist = hstack([features_by_dist, groups_one_hot_test])
    features_by_dist = features_by_dist.tocsr()
else:
    features_by_dist = get_features(train_data)
    features_by_dist = hstack([features_by_dist, groups_one_hot_train])
    features_by_dist = features_by_dist.tocsr()

In [22]:
target = train_data.target.values

NameError: name 'train_data' is not defined

In [ ]:
#VALIDATION
max_score = 0
scores = []
pca = PCA(n_components=4)
#j = pca.fit_transform(features_by_dist.toarray())
j = features_by_dist
for i in tqdm(range(5)):
    for train, test in get_splits(train_data,n=3,random_state=i):
        model = RandomForestClassifier(n_jobs=6,n_estimators=1000)
#        model = GradientBoostingClassifier(n_estimators=2000, learning_rate=0.002, max_depth=9)
#        model = LogisticRegression(n_jobs=6)pca = PCA(n_components=2)
#        model.fit(features_by_dist[train], target[train])
        model.fit(j[train], target[train])
        pred = model.predict(j[test])               
        score = f1_score(pred,train_data.target[test])
        scores.append(score)
        print(score, end=' ')
    print()

print('\nAvg score:' ,sum(scores)/len(scores))

In [ ]:
#SAVE model
from sklearn.ensemble import RandomForestClassifier 
model = RandomForestClassifier(n_jobs=6,n_estimators=1000)
model.fit(features_by_dist, target)
save_obj(model, 'model')

In [29]:
#SUBMIT
from sklearn.ensemble import RandomForestClassifier 
print(type(features_by_dist))
model = load_obj('model') 
pred = model.predict(features_by_dist)               
pred.shape

<class 'scipy.sparse.csr.csr_matrix'>


(16627,)

In [30]:
test_data['target'] = pred
test_data.to_csv('sub_with_meta',index=False,columns=['pair_id','target'])

In [ ]:
from time import sleep
gr = int(input())
print(gr)
pca = PCA(n_components=components=2)
i = pca.fit_transform(features_by_dist[train_data[train_data.group_id==gr].index.values].toarray())
a = plt.subplot()
a.scatter(i[:, 0], i[:, 1], c=target[train_data[train_data.group_id==gr].index.values])

In [ ]:
save_obj(test_data,'test')